# (FD22-0493) 聊天機械人的技術原理和應用


1. 瞭解人工智能發展的現況
2. 學習NLP自然語言的理論
3. 認識聊天機械人的技術和應用範圍
4. 利用Python語言實作簡單的聊天機械人




# 1.下載教材


In [1]:
!git clone https://github.com/pcmsricky/chatbot
%cd chatbot

Cloning into 'chatbot'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 27 (delta 0), reused 1 (delta 0), pack-reused 22
Unpacking objects: 100% (27/27), done.
/content/chatbot


# 2. Python 中的變數

In [ ]:
#單獨的變數
num = 3           #數字變數
text = 'Hello'    #文字變數

print(num)

In [ ]:
#list, 一個變數可以儲存多個資料
nlist = [1,3,5]
nlist.append(8)

tlist = ['hello','morning']
xlist = ['hello',1,4,'Peter']

print(nlist)

[1, 3, 5, 8]


# 3. 在list中搜尋

In [ ]:
DB = [1,4,5,8]
target = 4

for x in DB:
  if x==target:
    print('I found it.')
    break
else:
  print('not found.')

I found it.


# 4. 用list做語料庫

In [ ]:
DB = ['hello','morning','sunday','What is your name?']
target = input('youAsk:')
for q in DB:
  if q==target:
    print('I found it.')
    break
else:
  print('not found.')    

hello
I found it.


# 5.在list中搜尋最接近的值, 神奇的tuple

In [ ]:
DB = [1,4,5,8]
target = 4

tmp = []
for x in DB:
  tmp.append(  (abs(target-x),x) )

print(tmp)  
DB = sorted(tmp)
print(DB)
diff,ans = DB[0]
print(ans)

[(3, 1), (0, 4), (1, 5), (4, 8)]
[(0, 4), (1, 5), (3, 1), (4, 8)]
4


# 6. 自動回答機械人 ver#1

In [ ]:
DB = [('What is your name?','I am ricky.'),
      ('How old are you?','I am 18 years old.'),
      ('When does the class start?','The class starts at 8:05'),
      ('Where is puiching middle school?', 'Avenida de Horta e Costa, No.7, Macao.')
      ]

def match(alist,blist):
    count = 0
    for a in alist:
        if a in blist:
            count = count+1
    return count 

while (True):
    youAsk = input('youAsk: ')
    
    if youAsk=='bye':
        break
    else:
        askList = youAsk.split()
        nlist = []
        for q,a in DB:
            qList = q.split()
            c = match(askList,qList)
            nlist.append((c,a))
              
        nlist = sorted(nlist,reverse=True)
        c,a = nlist[0]
        print('bot:',a)

youAsk: what
bot: The class starts at 8:05
youAsk: What
bot: I am ricky.
youAsk: When
bot: The class starts at 8:05
youAsk: start
bot: The class starts at 8:05
youAsk: bye


## 安裝 jieba 中文斷字工具

In [ ]:
!pip install jieba

## 測試中文斷字

In [ ]:
import jieba
import logging
jieba.setLogLevel(logging.INFO)

askList = '你的名字是甚麼?'.split()
print(askList)

## 中文版的自動回答機械人 ver#2

In [ ]:
DB = [('你好嗎?', '我很好.'),
      ('你吃晚飯了嗎?', '吃了.'),
      ('你今年幾多歲?', '我今年18歲.'),
      ('你喜歡吃什麼?', '巧克力.')
      ]
	  

In [ ]:
import jieba
import logging
jieba.setLogLevel(logging.INFO)


def match(alist,blist):
    count = 0
    for a in alist:
        if a in blist:
            count = count+1
    return count 

def printDB():
    print('DB=[')
    for i in range(len(DB)-1):
        print(f'{DB[i]},')
    print(DB[len(DB)-1])    
    print(']') 

while (True):
    youAsk = input('你問: ')
    
    if youAsk=='再見':
        printDB()
        break
    else:
        askList = jieba.lcut(youAsk)
        nlist = []
        for q,a in DB:
            qList = jieba.lcut(q)
            c = match(askList,qList)
            nlist.append((c,a))
              
        nlist = sorted(nlist,reverse=True)
        c,a = nlist[0]
        if (c==0):
            new_answer=input('資料庫找不到對應的回答, 請教我如何回答:')
            DB.append((youAsk,new_answer))
        else:    
            print('機械人:',a)

你問: 你好
資料庫找不到對應的回答, 請教我如何回答:很好
你問: 你好
機械人: 很好
你問: 再見
DB=[
('What is your name?', 'I am ricky.'),
('How old are you?', 'I am 18 years old.'),
('When does the class start?', 'The class starts at 8:05'),
('Where is puiching middle school?', 'Avenida de Horta e Costa, No.7, Macao.'),
('你好', '很好')
]


In [2]:
!chmod +x ngrok
!./ngrok config add-authtoken 27jazIu23XBqR34opeYmKdPP9ma_45j9aVSbgCppXRWZy2gvC
get_ipython().system_raw('./ngrok http 8000 --log=stdout > ngrok.log  &')
!ps -ax

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ModuleNotFoundError: ignored

## 中文版的自動回答機械人 ver#3

In [4]:
with open("DB.txt", "r") as f:
    dataDB = f.read().rstrip()

DB = eval(dataDB)    

In [5]:
from flask import Flask, request, send_from_directory
from ipywidgets import widgets
from IPython.display import display
import io
import jieba, json
from werkzeug.serving import run_simple



def match(alist,blist):
    cout = 0
    for a in alist:
        if a in blist:
            cout = cout+1
    return cout 

def printDB():
    print('DB=[')
    for i in range(len(DB)-1):
        print(f'{DB[i]},')
    print(DB[len(DB)-1])    
    print(']') 

app = Flask("AI chat")


@app.route("/")
def form():
    with open('chatbot.html', 'r',encoding='utf8') as chatbotfile:
        data = chatbotfile.read()
    return data

    
@app.route("/ask", methods=['POST', 'GET'])
def ask():
    if not request.args.get('text'):
        return '都沒有文字:<';
    
    youAsk = request.args.get('text')
    print('有人問: %s'%youAsk)
    
    askList = jieba.lcut(youAsk)
    nlist = []
    for q,a in DB:
        qList = jieba.lcut(q)
        c = match(askList,qList)
        nlist.append((c,a))

    nlist = sorted(nlist,reverse=True)
    c,a = nlist[0]
    if (c==0):
        new_answer=input('資料庫找不到對應的回答, 請教我如何回答:')
        DB.append((youAsk,new_answer))
        response = new_answer
    else:    
        response = a
   
    print('機械人:',response)
    return response



if __name__ == "__main__":
    run_simple('localhost', 8000, app)

 * Running on http://localhost:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2022 03:20:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/May/2022 03:20:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 03:20:29] "GET /static/chatbot.ico HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 03:20:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 03:20:48] "GET /static/chatbot.ico HTTP/1.1" 200 -
Building prefix dict from the default dictionary ...


有人問: 你好


Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.117 seconds.
Prefix dict has been built successfully.
127.0.0.1 - - [17/May/2022 03:21:03] "GET /ask?text=你好 HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 03:21:03] "GET /static/typing2.gif HTTP/1.1" 200 -


機械人: 我很好,不用擔心.


# 自動回答的改進

## 「的的了了」不必計算

In [7]:
dropList = ['的','了']
youAsk = '你吃晚飯了嗎?'
askList = jieba.lcut(youAsk)
#askList = [x for x in askList if x not in dropList]
print(askList)

['你', '吃', '晚飯', '嗎', '?']


## 設定最低的匹配值


In [10]:
nlist = [(2,'a'),(1,'b'),(0,'c')]
therehold = 3
nlist = [x for x in nlist if x[0]>therehold]
nlist = sorted(nlist,reverse=True)
if len(nlist)==0 or nlist[0][0]<0:
  print('not found')
else:
  print(nlist[0])

not found


## 挑戰圖靈測試

In [16]:
import random
nlist = [(2,'a'),(2,'b'),(2,'c'),(1,'d')]
nlist = sorted(nlist,reverse=True)
maxi = nlist[0][0]
nlist = [x for x in nlist if x[0]==maxi]
random.shuffle(nlist)
print(nlist[0])

(2, 'b')


## 上下文處理

設定主謂賓語 或 時間、地點、人物等欄位, 下文未有提供的訊息, 用上文的補上

## 分佈式處理

設定多個主題自動回答機機械人, 利用分佈式網絡協同處理

## 將問題交比網絡自動回答機械人

In [17]:
import requests
s = '你好嗎?'
resp = requests.get("http://api.qingyunke.com/api.php", {'key': 'free', 'appid':0, 'msg': s})
resp.encoding = 'utf8'
resp = resp.json()
print('菲菲：', resp['content'])


菲菲： 我不好，你都不愛我，ToT
